Baseball Prediction: 5a - Getting (Raw) Individual Pitcher Data

    - In the previous notebook, we compared our simple, hitting-only model to the Las Vegas odds. We concluded that incorporating the starting pitcher information would be a crucial next step to improve our model.

    - In this notebook we will learn how to scrape individual, game-level, pitching data from retrosheet. We will write a loop to go through and download the data. This will enable us to augment our game-level dataframe with features derived from the previous performance of the starting pitcher.

    - Let's start by going to retrosheet and finding the stats for CC Sabathia (one of my favorite pitchers from my childhood).

www.retrosheet.org

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

import lxml
import html5lib
from urllib.request import urlopen
import time

from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://www.retrosheet.org/boxesetc/2007/Ksabac0010072007.htm'
page = requests.get(url)

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0//EN" "https://www.w3.org/TR/REC-html40/strict.dtd">

<html dir="LTR" lang="EN">
<pre><a href="../MISC/Kdescr.htm">Read Me</a></pre>
<head>
<title>The 2007 CLE A Regular Season Pitching Log for CC Sabathia</title>
<link href="https://www.retrosheet.org/menubar/menubar.css" rel="stylesheet" type="text/css"/>
<script src="https://www.retrosheet.org/menubar/menubar.js" type="text/javascript"></script>
</head>
<body>
<p class="nopad"><a href="https://www.retrosheet.org"><img alt="Retrosheet" class="bancenter" height="46" src="https://www.retrosheet.org/menubar/retro-logo.gif" width="400"/></a></p>
<div class="mbcenter">
<ul class="nav">
<li><a href="https://www.retrosheet.org/">Home</a>
<li><a href="https://www.retrosheet.org/searches/search.html">Search</a></li>
<li><a href="#">Games/People/Parks ↓</a>
<ul>
<li><a href="#">People →</a>
<ul>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Players">Players</a>
<li><a href="https://www.r

In [4]:
soup1 = list(soup.children)[-1]
soup1

<html dir="LTR" lang="EN">
<pre><a href="../MISC/Kdescr.htm">Read Me</a></pre>
<head>
<title>The 2007 CLE A Regular Season Pitching Log for CC Sabathia</title>
<link href="https://www.retrosheet.org/menubar/menubar.css" rel="stylesheet" type="text/css"/>
<script src="https://www.retrosheet.org/menubar/menubar.js" type="text/javascript"></script>
</head>
<body>
<p class="nopad"><a href="https://www.retrosheet.org"><img alt="Retrosheet" class="bancenter" height="46" src="https://www.retrosheet.org/menubar/retro-logo.gif" width="400"/></a></p>
<div class="mbcenter">
<ul class="nav">
<li><a href="https://www.retrosheet.org/">Home</a>
<li><a href="https://www.retrosheet.org/searches/search.html">Search</a></li>
<li><a href="#">Games/People/Parks ↓</a>
<ul>
<li><a href="#">People →</a>
<ul>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Players">Players</a>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Managers">Managers</a>
<li><a href="https://www.retrosheet.org/

In [5]:
soup2 = list(soup1.children)[-1]
soup2

<body>
<p class="nopad"><a href="https://www.retrosheet.org"><img alt="Retrosheet" class="bancenter" height="46" src="https://www.retrosheet.org/menubar/retro-logo.gif" width="400"/></a></p>
<div class="mbcenter">
<ul class="nav">
<li><a href="https://www.retrosheet.org/">Home</a>
<li><a href="https://www.retrosheet.org/searches/search.html">Search</a></li>
<li><a href="#">Games/People/Parks ↓</a>
<ul>
<li><a href="#">People →</a>
<ul>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Players">Players</a>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Managers">Managers</a>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Coaches">Coaches</a>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Umpires">Umpires</a>
<li><a href="https://www.retrosheet.org/transactions/index.html">Transactions</a>
</li></li></li></li></li></ul>
<li><a href="#">Games →</a>
<ul>
<li><a href="https://www.retrosheet.org/boxesetc/index.html">Regular season</a>
<li><a h

In [6]:
soup3 = list(soup2.children)
soup3

['\n',
 <p class="nopad"><a href="https://www.retrosheet.org"><img alt="Retrosheet" class="bancenter" height="46" src="https://www.retrosheet.org/menubar/retro-logo.gif" width="400"/></a></p>,
 '\n',
 <div class="mbcenter">
 <ul class="nav">
 <li><a href="https://www.retrosheet.org/">Home</a>
 <li><a href="https://www.retrosheet.org/searches/search.html">Search</a></li>
 <li><a href="#">Games/People/Parks ↓</a>
 <ul>
 <li><a href="#">People →</a>
 <ul>
 <li><a href="https://www.retrosheet.org/boxesetc/index.html#Players">Players</a>
 <li><a href="https://www.retrosheet.org/boxesetc/index.html#Managers">Managers</a>
 <li><a href="https://www.retrosheet.org/boxesetc/index.html#Coaches">Coaches</a>
 <li><a href="https://www.retrosheet.org/boxesetc/index.html#Umpires">Umpires</a>
 <li><a href="https://www.retrosheet.org/transactions/index.html">Transactions</a>
 </li></li></li></li></li></ul>
 <li><a href="#">Games →</a>
 <ul>
 <li><a href="https://www.retrosheet.org/boxesetc/index.html">R

In [7]:
index_num = np.where(["Opponent" in str(x) for x in soup3])[0][0]
index_num

12

In [8]:
soup4 = soup3[index_num]
soup4

<pre>   Date    #         Opponent  GS  CG SHO  GF  SV  IP     H  BFP  HR   R  ER  BB  IB  SO  SH  SF  WP HBP  BK  2B  3B GDP ROE   W   L    ERA
<a href="../2007/04022007.htm"> 4- 2-2007</a>   <a href="../2007/B04020CHA2007.htm">BOX+PBP</a> AT CHI A   1   0   0   0   0   6     8   26   2   3   3   1   0   2   0   0   0   0   0   1   0   1   0   1   0   4.50
<a href="../2007/04102007.htm"> 4-10-2007</a>   <a href="../2007/B04100CLE2007.htm">BOX+PBP</a> VS ANA A   1   0   0   0   0   7    10   31   0   3   1   2   0   7   0   0   0   0   0   1   0   2   0   1   0   2.77
<a href="../2007/04152007.htm"> 4-15-2007</a>   <a href="../2007/B04150CLE2007.htm">BOX+PBP</a> VS CHI A   1   0   0   0   0   8     5   32   0   1   1   3   0  10   1   0   0   0   0   0   0   0   0   1   0   2.14
<a href="../2007/04202007.htm"> 4-20-2007</a>   <a href="../2007/B04200TBA2007.htm">BOX+PBP</a> AT TB  A   1   0   0   0   0   7     7   27   1   3   2   1   0   8   0   0   0   1   0   3   0   1   0   0   0   

In [9]:
soup5 = list(soup4.children)
soup5

['   Date    #         Opponent  GS  CG SHO  GF  SV  IP     H  BFP  HR   R  ER  BB  IB  SO  SH  SF  WP HBP  BK  2B  3B GDP ROE   W   L    ERA\n',
 <a href="../2007/04022007.htm"> 4- 2-2007</a>,
 '   ',
 <a href="../2007/B04020CHA2007.htm">BOX+PBP</a>,
 ' AT CHI A   1   0   0   0   0   6     8   26   2   3   3   1   0   2   0   0   0   0   0   1   0   1   0   1   0   4.50\n',
 <a href="../2007/04102007.htm"> 4-10-2007</a>,
 '   ',
 <a href="../2007/B04100CLE2007.htm">BOX+PBP</a>,
 ' VS ANA A   1   0   0   0   0   7    10   31   0   3   1   2   0   7   0   0   0   0   0   1   0   2   0   1   0   2.77\n',
 <a href="../2007/04152007.htm"> 4-15-2007</a>,
 '   ',
 <a href="../2007/B04150CLE2007.htm">BOX+PBP</a>,
 ' VS CHI A   1   0   0   0   0   8     5   32   0   1   1   3   0  10   1   0   0   0   0   0   0   0   0   1   0   2.14\n',
 <a href="../2007/04202007.htm"> 4-20-2007</a>,
 '   ',
 <a href="../2007/B04200TBA2007.htm">BOX+PBP</a>,
 ' AT TB  A   1   0   0   0   0   7     7   27   1  

In [10]:
for i in range(12):
    print(soup5[i].get_text().split())

['Date', '#', 'Opponent', 'GS', 'CG', 'SHO', 'GF', 'SV', 'IP', 'H', 'BFP', 'HR', 'R', 'ER', 'BB', 'IB', 'SO', 'SH', 'SF', 'WP', 'HBP', 'BK', '2B', '3B', 'GDP', 'ROE', 'W', 'L', 'ERA']
['4-', '2-2007']
[]
['BOX+PBP']
['AT', 'CHI', 'A', '1', '0', '0', '0', '0', '6', '8', '26', '2', '3', '3', '1', '0', '2', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '4.50']
['4-10-2007']
[]
['BOX+PBP']
['VS', 'ANA', 'A', '1', '0', '0', '0', '0', '7', '10', '31', '0', '3', '1', '2', '0', '7', '0', '0', '0', '0', '0', '1', '0', '2', '0', '1', '0', '2.77']
['4-15-2007']
[]
['BOX+PBP']


In [11]:
## Given the url that refers to a specific pitcher and season
## we scrape the data and process it a bit

def get_season_pitching_data(url):    
    time.sleep(1)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    html=list(soup.children)[-1]
    body = list(html.children)[-1]
    sec_next = list(body.children)
    secnum = np.where(["Opponent" in str(x) for x in sec_next])[0][0]
    key_section = sec_next[secnum]
    working_part = list(key_section.children)
    p_header = working_part[0].strip().split()
    mod_header= ['at_vs','Opponent','League', 'GS', 'CG', 'SHO', 'GF', 'SV', 'IP', 'H',
            'BFP', 'HR', 'R', 'ER', 'BB', 'IB', 'SO', 'SH', 'SF', 'WP', 'HBP',
            'BK', '2B', '3B', 'GDP', 'ROE', 'W', 'L', 'ERA']

    date_list = []
    day_href_list = []
    for k in range(1,len(working_part),4):
        date_list.append(working_part[k].get_text().strip())
        day_href_list.append(working_part[k].attrs['href'])

    dblhead_num_list = []
    for k in range(2,len(working_part),4):
        dblhead_num_list.append(working_part[k].strip())

    game_href_list = []
    for k in range(3,len(working_part),4):
        game_href_list.append(working_part[k].attrs['href'])

    main_data_matrix = []
    for k in range(4,len(working_part),4):
        main_data_row = (working_part[k].strip().split())[:29]
        main_data_matrix.append(main_data_row)

    out_df = pd.DataFrame(main_data_matrix, columns = mod_header)
    out_df['Date'] = date_list
    out_df['dblhead_num'] = dblhead_num_list
    return(out_df)

In [12]:
get_season_pitching_data(url)

,at_vs,Opponent,League,GS,CG,SHO,GF,SV,IP,H,BFP,HR,R,ER,BB,IB,SO,SH,SF,WP,HBP,BK,2B,3B,GDP,ROE,W,L,ERA,Date,dblhead_num
0,AT,CHI,A,1,0,0,0,0,6,8,26,2,3,3,1,0,2,0,0,0,0,0,1,0,1,0,1,0,4.50,4- 2-2007,
1,VS,ANA,A,1,0,0,0,0,7,10,31,0,3,1,2,0,7,0,0,0,0,0,1,0,2,0,1,0,2.77,4-10-2007,
2,VS,CHI,A,1,0,0,0,0,8,5,32,0,1,1,3,0,10,1,0,0,0,0,0,0,0,0,1,0,2.14,4-15-2007,
3,AT,TB,A,1,0,0,0,0,7,7,27,1,3,2,1,0,8,0,0,0,1,0,3,0,1,0,0,0,2.25,4-20-2007,
4,VS,TEX,A,1,0,0,0,0,6,7,25,1,5,5,1,0,8,0,0,0,0,0,1,0,1,0,0,0,3.18,4-25-2007,
5,VS,TOR,A,1,0,0,0,0,6,4,22,3,3,3,0,0,9,0,0,0,0,0,0,0,0,0,1,0,3.38,5- 1-2007,
6,AT,BAL,A,1,0,0,0,0,7,9,30,0,3,3,1,0,9,0,0,0,0,0,2,0,0,0,1,0,3.45,5- 6-2007,
7,AT,OAK,A,1,0,0,0,0,6.2,9,29,2,6,6,1,0,5,0,0,0,0,0,3,0,0,0,0,1,4.02,5-11-2007,
8,VS,MIN,A,1,0,0,0,0,8,5,30,0,1,1,1,0,5,0,0,0,1,0,1,0,0,0,1,0,3.65,5-16-2007,
9,VS,SEA,A,1,0,0,0,0,5,8,22,0,1,1,1,0,4,0,0,0,1,0,2,0,0,0,0,0,3.51,5-21-2007,


In [13]:
url = 'https://www.retrosheet.org/boxesetc/S/Psabac001.htm'
page = requests.get(url)
sup = BeautifulSoup(page.content, 'html.parser')
sup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0//EN" "https://www.w3.org/TR/REC-html40/strict.dtd">

<html dir="LTR" lang="EN">
<pre><a href="../MISC/Pdescr.htm">Read Me</a></pre>
<head>
<title>CC Sabathia</title>
<link href="https://www.retrosheet.org/menubar/menubar.css" rel="stylesheet" type="text/css"/>
<script src="https://www.retrosheet.org/menubar/menubar.js" type="text/javascript"></script>
</head>
<body>
<p class="nopad"><a href="https://www.retrosheet.org"><img alt="Retrosheet" class="bancenter" height="46" src="https://www.retrosheet.org/menubar/retro-logo.gif" width="400"/></a></p>
<div class="mbcenter">
<ul class="nav">
<li><a href="https://www.retrosheet.org/">Home</a>
<li><a href="https://www.retrosheet.org/searches/search.html">Search</a></li>
<li><a href="#">Games/People/Parks ↓</a>
<ul>
<li><a href="#">People →</a>
<ul>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Players">Players</a>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Managers">Man

In [14]:
sup2 = list(sup.children)[2]
sup2

<html dir="LTR" lang="EN">
<pre><a href="../MISC/Pdescr.htm">Read Me</a></pre>
<head>
<title>CC Sabathia</title>
<link href="https://www.retrosheet.org/menubar/menubar.css" rel="stylesheet" type="text/css"/>
<script src="https://www.retrosheet.org/menubar/menubar.js" type="text/javascript"></script>
</head>
<body>
<p class="nopad"><a href="https://www.retrosheet.org"><img alt="Retrosheet" class="bancenter" height="46" src="https://www.retrosheet.org/menubar/retro-logo.gif" width="400"/></a></p>
<div class="mbcenter">
<ul class="nav">
<li><a href="https://www.retrosheet.org/">Home</a>
<li><a href="https://www.retrosheet.org/searches/search.html">Search</a></li>
<li><a href="#">Games/People/Parks ↓</a>
<ul>
<li><a href="#">People →</a>
<ul>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Players">Players</a>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Managers">Managers</a>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Coaches">Coaches</a>
<li><a

In [15]:
sup3 = list(sup2.children)[5]
sup3

<body>
<p class="nopad"><a href="https://www.retrosheet.org"><img alt="Retrosheet" class="bancenter" height="46" src="https://www.retrosheet.org/menubar/retro-logo.gif" width="400"/></a></p>
<div class="mbcenter">
<ul class="nav">
<li><a href="https://www.retrosheet.org/">Home</a>
<li><a href="https://www.retrosheet.org/searches/search.html">Search</a></li>
<li><a href="#">Games/People/Parks ↓</a>
<ul>
<li><a href="#">People →</a>
<ul>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Players">Players</a>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Managers">Managers</a>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Coaches">Coaches</a>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Umpires">Umpires</a>
<li><a href="https://www.retrosheet.org/transactions/index.html">Transactions</a>
</li></li></li></li></li></ul>
<li><a href="#">Games →</a>
<ul>
<li><a href="https://www.retrosheet.org/boxesetc/index.html">Regular season</a>
<li><a h

In [16]:
# Plan - find the <pre> tag that starts with 'Pitching Record' (after stripping whitespace)
# Get the href attribute for all the <a> tags with the word "Daily"

pre_tags = [x for x in sup3.find_all('pre')]
pre_tag_text = [x.get_text().strip() for x in pre_tags]
pre_tag_text

['Top Performances',
 'Pitcher Matchups   Batter Matchups',
 'Batting Record\nYear Team                     G    AB    R    H  2B  3B  HR  RBI   BB IBB   SO HBP  SH  SF  XI ROE GDP   SB  CS   AVG   OBP   SLG   BFW Year Team\n2001 CLE A    Daily Splits   33     4    0    0   0   0   0    0    0   0    0   0   1   0   0   0   1    0   0  .000  .000  .000   0.0 2001 CLE A\n2002 CLE A    Daily Splits   33     5    0    1   0   0   0    0    1   0    1   0   0   0   0   0   0    0   0  .200  .333  .200   0.0 2002 CLE A\n2003 CLE A    Daily Splits   31     6    1    3   0   0   0    0    0   0    1   0   0   0   0   0   0    0   0  .500  .500  .500   0.0 2003 CLE A\n2004 CLE A    Daily Splits   30     4    0    1   0   0   0    0    0   0    2   0   0   0   0   0   0    0   0  .250  .250  .250   0.0 2004 CLE A\n2005 CLE A    Daily Splits   31     6    1    2   1   0   1    4    0   0    1   0   0   0   0   0   0    0   0  .333  .333 1.000   0.0 2005 CLE A\n2006 CLE A    Daily Splits   28    

In [17]:
np.where([x.startswith('Pitching Record') for x in pre_tag_text])[0][0]

7

In [18]:
ind = np.where([x.startswith('Pitching Record') for x in pre_tag_text])[0][0]
a_tags = pre_tags[ind].find_all('a')
a_tags

[<a href="../2001/Y_2001.htm">2001</a>,
 <a href="../2001/TCLE02001.htm">CLE A</a>,
 <a href="../2001/Ksabac0010012001.htm">Daily</a>,
 <a href="../2001/Lsabac0010012001.htm">Splits</a>,
 <a href="../2001/Y_2001.htm">2001</a>,
 <a href="../2001/TCLE02001.htm">CLE A</a>,
 <a href="../2002/Y_2002.htm">2002</a>,
 <a href="../2002/TCLE02002.htm">CLE A</a>,
 <a href="../2002/Ksabac0010022002.htm">Daily</a>,
 <a href="../2002/Lsabac0010022002.htm">Splits</a>,
 <a href="../2002/Y_2002.htm">2002</a>,
 <a href="../2002/TCLE02002.htm">CLE A</a>,
 <a href="../2003/Y_2003.htm">2003</a>,
 <a href="../2003/TCLE02003.htm">CLE A</a>,
 <a href="../2003/Ksabac0010032003.htm">Daily</a>,
 <a href="../2003/Lsabac0010032003.htm">Splits</a>,
 <a href="../2003/Y_2003.htm">2003</a>,
 <a href="../2003/TCLE02003.htm">CLE A</a>,
 <a href="../2004/Y_2004.htm">2004</a>,
 <a href="../2004/TCLE02004.htm">CLE A</a>,
 <a href="../2004/Ksabac0010042004.htm">Daily</a>,
 <a href="../2004/Lsabac0010042004.htm">Splits</a>,


In [19]:
links = [x.attrs['href'] for x in a_tags if x.get_text()=='Daily']
links

['../2001/Ksabac0010012001.htm',
 '../2002/Ksabac0010022002.htm',
 '../2003/Ksabac0010032003.htm',
 '../2004/Ksabac0010042004.htm',
 '../2005/Ksabac0010052005.htm',
 '../2006/Ksabac0010062006.htm',
 '../2007/Ksabac0010072007.htm',
 '../2008/Ksabac0010082008.htm',
 '../2008/Ksabac0010092008.htm',
 '../2009/Ksabac0010102009.htm',
 '../2010/Ksabac0010112010.htm',
 '../2011/Ksabac0010122011.htm',
 '../2012/Ksabac0010132012.htm',
 '../2013/Ksabac0010142013.htm',
 '../2014/Ksabac0010152014.htm',
 '../2015/Ksabac0010162015.htm',
 '../2016/Ksabac0010172016.htm',
 '../2017/Ksabac0010182017.htm',
 '../2018/Ksabac0010192018.htm',
 '../2019/Ksabac0010202019.htm']

In [20]:
### Get the links to the pitcher-season tables given the pitcher id

def get_daily_season_links(pitcher_id):
    letter = pitcher_id.upper()[0]
    url_prefix = 'https://www.retrosheet.org/boxesetc/'
    url = url_prefix+letter+'/P'+pitcher_id+'.htm'
    time.sleep(1)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    html=list(soup.children)
    body = list(html[2].children)[5]
    pre_texts = [x for x in body.find_all('pre')]
    secnum = np.where([x.get_text().strip().startswith('Pitching Record') for x in pre_texts])[0][0]
    a_pre_texts = pre_texts[secnum].find_all('a')
    daily_season_links = [url_prefix+x.attrs['href'][3:] for x in a_pre_texts if x.get_text()=='Daily']
    return(daily_season_links)

In [21]:
get_daily_season_links('sabac001')

['https://www.retrosheet.org/boxesetc/2001/Ksabac0010012001.htm',
 'https://www.retrosheet.org/boxesetc/2002/Ksabac0010022002.htm',
 'https://www.retrosheet.org/boxesetc/2003/Ksabac0010032003.htm',
 'https://www.retrosheet.org/boxesetc/2004/Ksabac0010042004.htm',
 'https://www.retrosheet.org/boxesetc/2005/Ksabac0010052005.htm',
 'https://www.retrosheet.org/boxesetc/2006/Ksabac0010062006.htm',
 'https://www.retrosheet.org/boxesetc/2007/Ksabac0010072007.htm',
 'https://www.retrosheet.org/boxesetc/2008/Ksabac0010082008.htm',
 'https://www.retrosheet.org/boxesetc/2008/Ksabac0010092008.htm',
 'https://www.retrosheet.org/boxesetc/2009/Ksabac0010102009.htm',
 'https://www.retrosheet.org/boxesetc/2010/Ksabac0010112010.htm',
 'https://www.retrosheet.org/boxesetc/2011/Ksabac0010122011.htm',
 'https://www.retrosheet.org/boxesetc/2012/Ksabac0010132012.htm',
 'https://www.retrosheet.org/boxesetc/2013/Ksabac0010142013.htm',
 'https://www.retrosheet.org/boxesetc/2014/Ksabac0010152014.htm',
 'https://

In [22]:
get_season_pitching_data(get_daily_season_links('sabac001')[4])

,at_vs,Opponent,League,GS,CG,SHO,GF,SV,IP,H,BFP,HR,R,ER,BB,IB,SO,SH,SF,WP,HBP,BK,2B,3B,GDP,ROE,W,L,ERA,Date,dblhead_num
0,VS,MIN,A,1,0,0,0,0,5.2,5,22,0,1,1,1,0,7,0,0,0,0,0,0,0,1,0,0,0,1.59,4-17-2005,
1,AT,SEA,A,1,0,0,0,0,7,5,27,0,1,1,2,0,5,0,1,0,0,0,2,0,1,0,1,0,1.42,4-22-2005,
2,VS,KC,A,1,0,0,0,0,7,3,26,0,0,0,1,0,4,0,0,0,1,0,1,0,0,0,1,0,0.92,4-29-2005,
3,AT,MIN,A,1,0,0,0,0,4.1,9,27,1,5,5,4,0,0,0,0,0,1,0,1,0,0,0,0,1,2.62,5- 5-2005,
4,AT,ANA,A,1,0,0,0,0,6.1,8,26,2,4,4,1,0,5,0,0,0,0,0,1,0,0,0,0,1,3.26,5-10-2005,
5,VS,TOR,A,1,0,0,0,0,7.2,8,32,2,5,4,2,0,5,0,0,0,1,0,0,0,2,1,0,1,3.55,5-15-2005,
6,AT,CIN,N,1,0,0,0,0,6,7,24,0,3,2,0,0,6,0,0,0,0,0,2,0,1,0,1,0,3.48,5-21-2005,
7,VS,MIN,A,1,0,0,0,0,8.2,6,36,0,4,4,5,0,6,0,0,0,0,0,1,0,2,1,0,0,3.59,5-26-2005,
8,AT,MIN,A,1,0,0,0,0,7.2,7,31,0,3,3,2,0,3,0,0,0,0,0,2,0,1,0,1,0,3.58,5-31-2005,
9,AT,CHI,A,1,0,0,0,0,6.2,5,28,0,2,2,1,0,4,0,0,0,2,0,0,0,0,0,0,0,3.49,6- 5-2005,


In [23]:
# Get all the data for a particular pitcher
def get_full_pitching_data(pitcher_id):
    link_list = get_daily_season_links(pitcher_id)
    df_pitching = pd.DataFrame()
    for url in link_list:
        df_pitching = pd.concat((df_pitching, get_season_pitching_data(url)))
    return(df_pitching)

In [24]:
cc_data = get_full_pitching_data('sabac001')

In [25]:
cc_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 561 entries, 0 to 22
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   at_vs        561 non-null    object
 1   Opponent     561 non-null    object
 2   League       561 non-null    object
 3   GS           561 non-null    object
 4   CG           561 non-null    object
 5   SHO          561 non-null    object
 6   GF           561 non-null    object
 7   SV           561 non-null    object
 8   IP           561 non-null    object
 9   H            561 non-null    object
 10  BFP          561 non-null    object
 11  HR           561 non-null    object
 12  R            561 non-null    object
 13  ER           561 non-null    object
 14  BB           561 non-null    object
 15  IB           561 non-null    object
 16  SO           561 non-null    object
 17  SH           561 non-null    object
 18  SF           561 non-null    object
 19  WP           561 non-null    o

In [26]:
cc_data.sample(5)

,at_vs,Opponent,League,GS,CG,SHO,GF,SV,IP,H,BFP,HR,R,ER,BB,IB,SO,SH,SF,WP,HBP,BK,2B,3B,GDP,ROE,W,L,ERA,Date,dblhead_num
7,AT,BAL,A,1,0,0,0,0,6,8,30,1,4,4,4,0,6,1,0,0,1,0,3,0,1,0,0,1,3.77,5-15-2012,
14,VS,SD,N,1,0,0,0,0,8,6,30,0,3,3,1,0,10,0,1,0,0,0,2,0,0,0,1,0,4.26,6-15-2008,
26,AT,MIN,A,1,0,0,0,0,8,6,31,0,2,2,1,0,10,0,0,0,0,0,0,0,0,0,1,0,3.42,9-26-2012,
17,VS,NY,A,1,0,0,0,0,7,5,32,0,4,2,6,0,6,0,0,0,0,0,1,0,1,1,0,1,3.23,7- 9-2003,
26,AT,DET,A,1,1,0,0,0,9,4,30,1,1,1,0,0,10,0,0,0,0,0,0,0,1,0,1,0,4.35,9- 7-2005,


LOAD IN GAME LEVEL DATA

In [27]:
df = pd.read_csv('df_bp3.csv', low_memory=False)

In [28]:
start_pitchers_h = df.pitcher_start_id_h.unique()
start_pitchers_v = df.pitcher_start_id_v.unique()
len(start_pitchers_h), len(start_pitchers_v)

(3184, 3213)

In [29]:
start_pitchers_all = np.union1d(start_pitchers_h.astype(str), start_pitchers_v.astype(str))
len(start_pitchers_all), start_pitchers_all[:25]

(3445,
 array(['aased001', 'abadf001', 'abboc001', 'abbog001', 'abboj001',
        'abbok001', 'abbop001', 'abrej001', 'aceva001', 'acevj001',
        'acevj002', 'ackej001', 'acose101', 'adama002', 'adamc002',
        'adamj001', 'adamt001', 'adamw001', 'adcon001', 'adenn001',
        'adkis001', 'adlet001', 'adonj001', 'affej001', 'agosj001'],
       dtype='<U8'))

In [30]:
# run this for everyone in the list - may take a bit to run... (This is the data I provide for you in SP_Data)
'''
for p_id in start_pitchers_all:
    print(p_id)
    try:
        df_temp = get_full_pitching_data(p_id)
    except (AttributeError, AssertionError, ValueError):
        pass

    fname_out = '/Users/antiprotons/Desktop/beatingVegas/SP_Data/pitching_data_'+p_id+'.csv'
    df_temp.to_csv(fname_out, index=False)
'''    

"\nfor p_id in start_pitchers_all:\n    print(p_id)\n    try:\n        df_temp = get_full_pitching_data(p_id)\n    except (AttributeError, AssertionError, ValueError):\n        pass\n\n    fname_out = '/Users/antiprotons/Desktop/beatingVegas/SP_Data/pitching_data_'+p_id+'.csv'\n    df_temp.to_csv(fname_out, index=False)\n"